In [1]:
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict
from tqdm.autonotebook import tqdm
#import pandas as pd
import mysql.connector
import codecs
codecs.register(lambda name: codecs.lookup('utf8') if name == 'utf8mb4' else None)
import conllu
from nltk import ngrams

In [3]:
import re

In [4]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd='password',
  #database="small_commits_db"
  database='commits_db',
  charset='utf8mb4')

mycursor = mydb.cursor()

db = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd='password',
  database='pruned_commits_db2',
  charset='utf8mb4')

cur = db.cursor()

In [4]:
k = 0
for line in open('frequency_dict_no_pos_proc.csv'):
    if k == 0:
        k += 1
    else:
        row = line.strip().split('\t')
        if row[0].isalpha():
            nmbrs = [int(i) for i in row[1:]]
            #print(len(nmbrs))
            try:
                cur.execute('INSERT INTO lemmas (lemma) VALUES (%s)', (row[0],))
            except:
                pass
db.commit()

In [5]:
lemma_dict = {}
cur.execute('SELECT * FROM lemmas')
for i in cur.fetchall():
    lemma_dict[i[1]] = i[0]

In [6]:
freq_dict = {}
seen = set()
k = 0
for line in open('frequency_dict_no_pos_proc.csv'):
    if k == 0:
        k += 1
    else:
        row = line.strip().split('\t')
        if row[0].isalpha() and row[0] not in seen and row[0] in lemma_dict:
            seen.add(row[0])
            nmbrs = [int(i) for i in row[1:]]
            #print(len(nmbrs))
            cur.execute('INSERT INTO frequency_dict \
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)',
                       [lemma_dict[row[0]]]+nmbrs)
db.commit()

### write data

In [5]:
grammar_dict = {}
grammar_count = 0

lex_gr_pair = {}
lex_gr_count = 0

langs = {}
cur.execute('SELECT id, language_name FROM languages')
for i in cur.fetchall():
    langs[i[1]] = i[0]

repos = set()
mycursor.execute('SELECT gh_id, language FROM R')
for i in mycursor.fetchall():
    if i[1] in langs:
        repos.add(i[0])

lemma_dict = {}
cur.execute('SELECT * FROM lemmas')
for i in cur.fetchall():
    lemma_dict[i[1]] = i[0]

len(repos), len(lemma_dict)

user_dict = {}
mycursor.execute('SELECT id, login FROM U')
for i in mycursor.fetchall():
    user_dict[i[1]] = i[0]

relations = {}
cur.execute('SELECT id, relation_name FROM relations')
for i in cur.fetchall():
    relations[i[1]] = i[0]

In [5]:
def unify_gr(feats):
    try:
        return '|'.join('='.join(i) for i in feats.items())
    except AttributeError:
        return ''

def yield_c():
    mycursor.execute('SELECT * FROM C;')
    item = mycursor.fetchone()
    #print (text)
    while item:
        yield item
        item = mycursor.fetchone()

example_count = 0

def parse_doc(item, global_key):
    global grammar_count, lex_gr_count, example_count, lemma_dict, grammar_dict, lex_gr_pair
    text = item[3]
    date = item[4][:10]
    if item[2] in repos and item[1] in user_dict:
        user = user_dict[item[1]]
        repo = item[2]
        doc = conllu.parse(item[-1])
        new_doc = []
        example_used = False
        to_append = []
        for sentence in doc:
            snt = [None]
            for token in sentence.tokens:
                if token['lemma'] in lemma_dict:
                    gram_str = unify_gr(token['feats'])
                    upos = token['upostag']
                    if (gram_str, upos,) in grammar_dict:
                        gram = grammar_dict[(gram_str, upos,)]
                    else:
                        cur.execute('INSERT INTO grammar (string_format, pos) VALUES (%s, %s)', (gram_str, upos,))
                        grammar_count += 1
                        gram = grammar_count
                        grammar_dict[(gram_str, upos,)] = gram

                    lemma = lemma_dict[token['lemma']]
                    if (lemma, gram,) not in lex_gr_pair:
                        cur.execute('INSERT INTO lemma_grammar_pairs (id_lemma, id_grammar, form) \
                        VALUES (%s, %s, %s)', (lemma, gram, token['form'].lower()))
                        lex_gr_count += 1
                        lex_gr_pair[(lemma, gram, )] = lex_gr_count
                    lg = lex_gr_pair[(lemma, gram,)]
                    snt.append((lemma, gram, lg, token['deprel'], token['head']))
                else:
                    snt.append(None)
            for key, token in enumerate(snt):
                if key > 0:
                    if token != None:
                        if snt[token[4]] is not None:
                            idx = snt[token[4]][2]
                            if not example_used:
                                cur.execute('INSERT INTO examples \
                                            (text, id_user, id_repo, date) \
                                            VALUES (%s, %s, %s, %s)', (text, user, repo, date,))
                                example_used = True
                                example_count += 1
                                example = example_count
                            diff = token[4]-key
                            if diff > 0: diff = min(diff, 126)
                            else: diff = max(diff, -126)
                            to_append.append((idx, token[2], relations[token[3]], example, diff),)
        cur.executemany('INSERT INTO relation_pair \
        (id_head, id_dependent, id_relation, id_example, idx_diff) \
        VALUES (%s, %s, %s, %s, %s)', to_append)
    if global_key % 1000 == 0:
        db.commit()

In [6]:
lex_gr_count_dict = defaultdict(int)

In [7]:
def unify_gr(feats):
    try:
        return '|'.join('='.join(i) for i in feats.items())
    except AttributeError:
        return ''

def yield_c():
    mycursor.execute('SELECT * FROM C;')
    item = mycursor.fetchone()
    #print (text)
    while item:
        yield item
        item = mycursor.fetchone()

example_count = 0

def parse_doc(item, global_key):
    global grammar_count, lex_gr_count, example_count, lemma_dict, grammar_dict, lex_gr_pair
    text = item[3]
    date = item[4][:10]
    if item[2] in repos and item[1] in user_dict:
        user = user_dict[item[1]]
        repo = item[2]
        doc = conllu.parse(item[-1])
        new_doc = []
        example_used = False
        to_append = []
        for sentence in doc:
            snt = [None]
            for token in sentence.tokens:
                if token['lemma'] in lemma_dict:
                    gram_str = unify_gr(token['feats'])
                    upos = token['upostag']
                    if (gram_str, upos,) in grammar_dict:
                        gram = grammar_dict[(gram_str, upos,)]
                    else:
                        cur.execute('INSERT INTO grammar (string_format, pos) VALUES (%s, %s)', (gram_str, upos,))
                        grammar_count += 1
                        gram = grammar_count
                        grammar_dict[(gram_str, upos,)] = gram

                    lemma = lemma_dict[token['lemma']]
                    if (lemma, gram,) not in lex_gr_pair:
                        cur.execute('INSERT INTO lemma_grammar_pairs (id_lemma, id_grammar, form) \
                        VALUES (%s, %s, %s)', (lemma, gram, token['form'].lower()))
                        lex_gr_count += 1
                        lex_gr_pair[(lemma, gram, )] = lex_gr_count
                    lg = lex_gr_pair[(lemma, gram,)]
                    lex_gr_count_dict[(lemma, gram,)]+=1
                    snt.append((lemma, gram, lg, token['deprel'], token['head']))
                else:
                    snt.append(None)
            for key, token in enumerate(snt):
                if key > 0:
                    if token != None:
                        if snt[token[4]] is not None:
                            idx = snt[token[4]][0]
                            if not example_used:
                                cur.execute('INSERT INTO examples \
                                            (text, id_user, id_repo, date) \
                                            VALUES (%s, %s, %s, %s)', (text, user, repo, date,))
                                example_used = True
                                example_count += 1
                                example = example_count
                            diff = token[4]-key
                            if diff > 0: diff = min(diff, 126)
                            else: diff = max(diff, -126)
                            to_append.append((idx, token[0], relations[token[3]], example, diff),)
        cur.executemany('INSERT INTO relation_pair \
        (id_head, id_dependent, id_relation, id_example, idx_diff) \
        VALUES (%s, %s, %s, %s, %s)', to_append)
    if global_key % 1000 == 0:
        db.commit()

In [8]:
cur.execute('DELETE FROM relation_pair')
cur.execute('ALTER TABLE relation_pair AUTO_INCREMENT = 1 ;')
cur.execute('DELETE FROM lemma_grammar_pairs')
cur.execute('ALTER TABLE lemma_grammar_pairs AUTO_INCREMENT = 1 ;')
cur.execute('DELETE FROM grammar')
cur.execute('ALTER TABLE grammar AUTO_INCREMENT = 1 ;')
cur.execute('DELETE FROM examples')
cur.execute('ALTER TABLE examples AUTO_INCREMENT = 1 ;')

db.commit()

In [9]:
for key, item in tqdm(enumerate(yield_c())): 
    try:
        parse_doc(item, key) 
    except Exception as e: print(e)

In [10]:
db.commit()

In [12]:
len(lex_gr_count_dict), len(lex_gr_pair)

(404717, 404717)

In [13]:
for key in lex_gr_pair:
    print(key, lex_gr_pair[key])
    break
for key in lex_gr_count_dict:
    print(key, lex_gr_count_dict[key])
    break

(5178, 60) 51702
(5178, 60) 39


In [14]:
data = []
for key in lex_gr_pair:
    data.append((lex_gr_count_dict[key], lex_gr_pair[key],))

In [15]:
data[:10]

[(39, 51702),
 (33, 196299),
 (3, 278001),
 (9, 184875),
 (1, 183253),
 (1, 358498),
 (5, 35800),
 (1, 307273),
 (8, 367695),
 (2, 193726)]

In [19]:
mx = 0
for i in data:
    mx = max(mx, i[0])
mx

18798193

In [21]:
cur.executemany('UPDATE lemma_grammar_pairs SET totalcount=%s WHERE id=%s', data)

In [22]:
db.commit()

In [ ]:
lex_gr_count_dict

In [20]:
db.rollback()

In [7]:
%load_ext line_profiler

In [8]:
for i in yield_c():
    data = i
    break

In [9]:
def run():
    for key, item in tqdm(enumerate(yield_c())): 
        parse_doc(item) 

In [10]:
%lprun -f parse_doc parse_doc(data, 1)

## id_lgp to id_lemma

In [1]:
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict
from tqdm.autonotebook import tqdm
#import pandas as pd
import mysql.connector
import codecs
codecs.register(lambda name: codecs.lookup('utf8') if name == 'utf8mb4' else None)
import conllu
from nltk import ngrams

In [2]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd='password',
  #database="small_commits_db"
  database='pruned_commits_db2',
  charset='utf8mb4')

mycursor = mydb.cursor()

db = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd='password',
  database='pruned_commits_db2',
  charset='utf8mb4')

cur = db.cursor()

In [9]:
def yield_c():
    mycursor.execute('SELECT id_head, id_relation, id_dependent, id_example, idx_diff FROM relation_pair')
    item = mycursor.fetchmany(1000)
    #print (text)
    while item:
        yield item
        item = mycursor.fetchmany(1000)

In [10]:
cur.execute('DELETE FROM examples_bi')
cur.execute('ALTER TABLE examples_bi AUTO_INCREMENT = 1 ;')

db.commit()

In [11]:
rel_pairs = {}
counter = 0
data = []
for key, batch in tqdm(enumerate(yield_c())): 
    for item in batch:
        try:
            if item[:3] not in rel_pairs:
                counter += 1
                rel_pairs[item[:3]] = counter
            data.append((rel_pairs[item[:3]], item[3], item[4],))
        except Exception as e: print(e)
    if key % 100 == 0:
        cur.executemany('INSERT INTO examples_bi (id_rel_pair, id_example, idx_diff) VALUES (%s, %s, %s)', data)
        db.commit()
        data = []

In [12]:
len(rel_pairs)

29965515

In [15]:
db.rollback()

In [17]:
data = []
for key, value in tqdm(enumerate(sorted(rel_pairs, key=rel_pairs.get))):
    if key % 100000 == 0:
        cur.executemany('INSERT INTO aggr_rel_pairs (id, id_head, id_relation, id_dependent) \
                                            VALUES (%s, %s, %s, %s)', data)
        db.commit()
        data = []
    else:
        data.append((rel_pairs[value], *value,))

In [18]:
cur.executemany('INSERT INTO aggr_rel_pairs (id, id_head, id_relation, id_dependent) \
                                            VALUES (%s, %s, %s, %s)', data)
db.commit()

In [3]:
def yield_re():
    mycursor.execute('SELECT id_rel_pair FROM examples_bi')
    item = mycursor.fetchmany(1000)
    #print (text)
    while item:
        yield item
        item = mycursor.fetchmany(1000)

In [4]:
rel_pairs = defaultdict(int)
for key, batch in tqdm(enumerate(yield_re())): 
    for item in batch:
        rel_pairs[item[0]] += 1

In [6]:
data = []
for key, value in tqdm(enumerate(sorted(rel_pairs))):
    if key % 100000 == 0:
        cur.executemany('UPDATE aggr_rel_pairs SET total_count=%s WHERE id=%s', data)
        db.commit()
        data = []
    else:
        data.append((rel_pairs[value], value, ))

In [8]:
cur.executemany('UPDATE aggr_rel_pairs SET total_count=%s WHERE id=%s', data)
db.commit()